# 10. Dataset Transforms

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [ ]:
class WineDataset(Dataset): # inheriting Dataset

  def __init__(self, transform=None):
    # data loading
    xy = np.loadtxt('/content/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
    self.n_samples = xy.shape[0]

    # note that we do not convert to tensor here
    self.x = xy[:, 1:]
    self.y = xy[:, [0]] # n_samples, 1
    
    self.transform = transform
  
  def __getitem__(self, index):
    sample = self.x[index], self.y[index]

    if self.transform:
      sample = self.transform(sample)
    
    return sample
  
  def __len__(self):
    return self.n_samples

In [ ]:
class ToTensor:
  def __call__(self, sample):
    inputs, targets = sample
    return torch.from_numpy(inputs), torch.from_numpy(targets)

In [ ]:
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
dataset = WineDataset(transform=None)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
class MulTransform:
  def __init__(self, factor):
    self.factor = factor
  
  def __call__(self, sample):
    inputs, targets = sample
    inputs *= self.factor
    return inputs, targets

In [ ]:
dataset = WineDataset(transform=None)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
print(features)

[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]


In [ ]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
print(features)

tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03])


# 11. Softmax and Crossentropy

In [ ]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print('softmax numpy:', outputs)

softmax numpy: [0.65900114 0.24243297 0.09856589]


In [ ]:
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)
print(outputs)

tensor([0.6590, 0.2424, 0.0986])


In [ ]:
# softmax loss is combined with crossentropy loss many times
def crossentropy(actual, predicted):
    loss = -np.sum(actual * np.log(predicted))
    return loss

In [ ]:
x = np.array([0.7, 0.2, 0.1])
y = np.array([1, 0, 0])
print(crossentropy(y, x))

0.35667494393873245


In [ ]:
x = np.array([0.1, 0.3, 0.6])
print(crossentropy(y, x))

2.3025850929940455


In [ ]:
# PYTORCH
import torch.nn as nn

loss = nn.CrossEntropyLoss()

Y = torch.tensor([0])
# nsamples x nclasses = 1x3
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])

In [ ]:
l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(l1.item())
print(l2.item())

0.4170299470424652
1.840616226196289


In [ ]:
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
print(predictions1)
print(predictions2)

tensor([0])
tensor([1])


Let's implement a simple neural net with a couple hidden layers that has a linear layer at the end, with one output for each class. This can be passed through softmax to get the probabilities.
In PyTorch, one must be careful because we use the cross entropy loss. Here, we must not use the softmax layer in the neural net!

In [ ]:
# multiclass problem
class NeuralNet2(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet2, self).__init__()
    self.linear1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self, x):
    out = self.linear1(x)
    out = self.relu(out)
    out = self.linear2(out)
    # no softmax at the end
    return out

In [ ]:
model = NeuralNet2(input_size=28*28, hidden_size=5, num_classes=3)
criterion = nn.CrossEntropyLoss() # (applies softmax)

Incase we want neural net to output yes or no answer (whether or not image is that of a dog), we can just change size of output of linear2 layer to 1. So, we have a linear layer with one output only. We can pass the output of this linear layer through sigmoid. We say yes if the sigmoid output is higher than 0.5. We also change loss function to BCELoss. Binary Cross Entropy Loss. Sigmoid has to be implemented at the end.

In [ ]:
# multiclass problem
class NeuralNet1(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(NeuralNet1, self).__init__()
    self.linear1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(hidden_size, 1)
  
  def forward(self, x):
    out = self.linear1(x)
    out = self.relu(out)
    out = self.linear2(out)
    # sigmoid at the end
    y_pred = torch.sigmoid(out)
    return y_pred

In [ ]:
model = NeuralNet1(input_size=28*28, hidden_size=5)
criterion = nn.BCELoss()

# 12. Activation Functions

In [ ]:
# import torch
# import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# option 1 (create nn module)
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(NeuralNet, self).__init__()
    self.linear1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(hidden_size, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.linear1(x)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.sigmoid(out)
    return out

In [ ]:
# option 2 (use activation function directly in forward pass)
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(NeuralNet, self).__init__()
    self.linear1 = nn.Linear(input_size, hidden_size)
    # self.relu = nn.ReLU()
    self.linear2 = nn.Linear(hidden_size, 1)
    # self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    # out = self.linear1(x)
    # out = self.relu(out)
    # out = self.linear2(out)
    # out = self.sigmoid(out)
    out = torch.relu(self.linear1(x))
    out = torch.sigmoid(self.linear2(out))
    return out